# Accessing Solr via [pysolr]()

## Downloading, unziping and starting Solr in cluster mode

In [ ]:
! wget  http://ftp.unicamp.br/pub/apache/lucene/solr/8.1.1/solr-8.1.1.zip -q
! unzip -qq solr-8.1.1.zip && rm solr-8.1.1.zip
! ./solr-8.1.1/bin/solr start -c

## Creating a collection

In [ ]:
! ./solr-8.1.1/bin/solr create -c cabot -p 8983

## Connecting to the Solr server

In [ ]:
import pysolr
solr_server_url = 'http://localhost:8983/solr/'
solr_collection = "cabot"
solr = pysolr.Solr(solr_server_url + solr_collection)

In [ ]:
import pandas as pd
cabot_book = pd.read_csv("../../data/case-teaching-cabot/case-teaching-cabot.csv")
cabot_book

In [ ]:
for index, row in cabot_book.iterrows():
    print(index, row['case'])

In [ ]:
for index, row in cabot_book.iterrows():
    solr.add([{
                "id": index+1,
                "case": row['case']
              }], commit=True)

In [ ]:
result = solr.search('*:*')
print(result.hits)
print(result['case'])

In [ ]:
results = solr.search('*:*')

print("Found {0} documents".format(len(results)))

for result in results:
    print("The document {0} has the following 'case': {1}".format(result['id'], result['case']))